In [1]:
from games_setup import *

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
simple = load_file(18)
simple.reactions

[SHMT: FH4 + serine -> CH2FH4; cell * (Vm * (serine / Km2 / (1 + serine / Km2)) * (FH4 / Km1) / (1 + FH4 / Km1)),
 SHMTr: CH2FH4 -> FH4; cell * (Vm * (glycine / Km2 / (1 + glycine / Km2)) * (CH2FH4 / Km1) / (1 + CH2FH4 / Km1)),
 HCHOtoCH2FH4: FH4 + HCHO -> CH2FH4; cell * hp * FH4 * HCHO,
 CH2FH4toHCHO: CH2FH4 -> FH4 + HCHO; cell * hl * CH2FH4,
 MTHFR: CH2FH4 + NADPH -> CH3FH4; cell * (Vm * CH2FH4 * NADPH / (NADPH * CH2FH4 + CH2FH4 * Km2 + (NADPH + Km2) * Km1 * (1 + MTX1 / Ki21 + MTX2 / Ki22 + MTX3 / Ki23 + MTX4 / Ki24 + MTX5 / Ki25 + FH2f / Ki1))),
 MTR: CH3FH4 + homocysteine -> FH4; cell * (Vm * (homocysteine / Km2 / (1 + homocysteine / Km2)) * (CH3FH4 / Km1) / (1 + CH3FH4 / Km1)),
 HCOOHtoCHOFH4: FH4 + formate + ATP -> CHOFH4; cell * (Vm / ((1 + Km1 / FH4) * (1 + Km2 / ATP) * (1 + Km3 / formate))),
 GARFT: CHOFH4 + GAR -> FGAR + FH4; cell * (Vm * CHOFH4 * GAR / (GAR * CHOFH4 + CHOFH4 * Km2 + (GAR + Km2) * Km1 * (1 + MTX1 / Ki21 + MTX2 / Ki22 + MTX3 / Ki23 + MTX4 / Ki24 + MTX5 / Ki25 

In [26]:
print(simple.molecules)

[FH4, serine, CH2FH4, HCHO, NADPH, CH3FH4, homocysteine, formate, ATP, CHOFH4, GAR, FGAR, AICAR, NADP, dUMP, FH2f, FFH2, MTX1, MTX2, MTX3, MTX4, MTX5, EMTX, DHFRf, MTX1b, MTX2b, MTX3b, MTX4b, MTX5b]


In [27]:
for reaction in simple.reactions:
    print(reaction.category)

reaction_n_1
reaction_1_1
reaction_n_1
reaction_1_n
reaction_n_1
reaction_n_1
reaction_n_1
reaction_n_n
reaction_n_1
reaction_n_1
reaction_n_1
reaction_1_1
reaction_1_1
reaction_n_1
reaction_1_1
reaction_1_1
reaction_1_1
reaction_1_1
reaction_1_1
reaction_1_1
reaction_1_1
reaction_1_1
reaction_1_1
reaction_1_1
reaction_boundary
reaction_boundary
reaction_boundary
reaction_boundary
reaction_boundary
reaction_n_1
reaction_n_1
reaction_n_1
reaction_n_1
reaction_n_1
reaction_1_n
reaction_1_n
reaction_1_n
reaction_1_n
reaction_1_n
reaction_boundary
reaction_boundary
reaction_boundary
reaction_1_1
reaction_1_1
reaction_1_1
reaction_1_1
reaction_1_1


In [6]:
from SBMLLint.games.som import SOM

In [28]:
# soms = []
# for mole in simple.molecules:
#     soms.append(SOM({mole}))
# #c = SOM({simple.molecules[0]})
# #m = SOM({simple.molecules[1]})
# print(soms)

In [8]:
# class Arc(object):
    
#     def __init__(self, source, destination, reaction):
#         """
#         :param SOM source:
#         :param SOM destination:
#         :param Reaction reaction:
#         """
#         self.source = source
#         self.destination = destination
#         self.reaction = reaction

In [29]:
""" The following can be used to add multiple entries to an edge

G = nx.DiGraph()
G.add_edges_from([('a', 'b'), ('b', 'c')])
#set_edge_attributes(G, name, values)
G.add_edge('d', 'e', reaction = [1,2])
print("first edge")
print(G.edges(data=True))
a = G.get_edge_data('d', 'e')['reaction']
G.add_edge('d', 'e', reaction = a + [3, 4])
print("second")
print(G.edges(data=True))
"""

' The following can be used to add multiple entries to an edge\n\nG = nx.DiGraph()\nG.add_edges_from([(\'a\', \'b\'), (\'b\', \'c\')])\n#set_edge_attributes(G, name, values)\nG.add_edge(\'d\', \'e\', reaction = [1,2])\nprint("first edge")\nprint(G.edges(data=True))\na = G.get_edge_data(\'d\', \'e\')[\'reaction\']\nG.add_edge(\'d\', \'e\', reaction = a + [3, 4])\nprint("second")\nprint(G.edges(data=True))\n'

In [22]:
#ARROW = "->"
REACTION = "reaction"
LESSTHAN = "<"

class MESGraph(nx.DiGraph):
  """
  The MESGraph class represents a collection of SOMs as nodes
  and their inequality relationships as edges (arcs). 
  Mass inequality between SOMs from reactions can help us
  detect their relationship.
  Type I Error occurs when we find inequality between two molecules
  in the same SOM, because each element in a SOM has the same weight.
  Type II Error implies there is cyclism between molecules, such as
  A < B < C < ... < A, which is physically impossible. 
  """

  def __init__(self, simple=None):
    """
    :param list-SOM soms:
    """
    super(MESGraph, self).__init__()
    self.soms = self.initializeSOMs(simple)
    self.add_nodes_from(self.soms)
    self.identifier = self.makeId()

  def __repr__(self):
    return self.identifier

  def initializeSOMs(self, simple):
    """
    Create a list of one-molecule SOMs
    :param SimpleSBML simple:
    :return list-SOM:
    """
    soms = []
    for molecule in simple.molecules:
      soms.append(SOM({molecule}))
    return soms
    
  def makeId(self):
    identifier = ""
    if self.edges:
      for edge in self.edges:
        identifier = identifier + str(edge[0]) + cn.ARC_ARROW + str(edge[1]) + "\n"

    for key, node in enumerate(nx.isolates(self)):
      identifier = identifier + str(node)
      if key < (len(list(nx.isolates(self)))-1):
        identifier = identifier + cn.KINETICS_SEPARATOR

    return identifier

  def getNode(self, molecule):
    """
    Search nodes in MESGraph and 
    returns SOM that includes the given molecule
    If no such SOM exists, return False
    :param Molecule molecule:
    :return SOM/False:
    """
    for som in list(self.nodes):
      for mole in som.molecules:
        if mole.name == molecule.name:
          return som
    return False

  def processUniUniReaction(self, reaction):
    """
    Process a 1-1 reaction to merge nodes.
    If no need to merge, return None.
    :param Reaction reactions:
    """
    if reaction.category != cn.REACTION_1_1:
      pass
    else:
      reactant_som = self.getNode(reaction.reactants[0].molecule)
      product_som = self.getNode(reaction.products[0].molecule)
      if reactant_som == product_som:
        return None
      else:
        new_som = reactant_som.merge(product_som)
        new_som.reactions.add(reaction)
        # TODO: if there are edges, need to also check them
        self.remove_node(reactant_som)
        self.remove_node(product_som)
        self.add_node(new_som)
        self.identifier = self.makeId()
        return new_som
    
  def processUniMultiReaction(self, reaction):
    """
    Process a 1-n reaction to add arcs.
    Since the mass of reactant is greater than
    that of each product, it adds arcs by
    addArc(source=products, destination=reactant). 
    :param Reaction reaction:
    """
    if reaction.category != cn.REACTION_1_n:
      pass
    else:
      destination = [reaction.reactants[0].molecule]
      source = [product.molecule for product in reaction.products]
      self.addArc(source, destination, reaction)
      self.identifier = self.makeId()

  def processMultiUniReaction(self, reaction):
    """
    Process a n-1 reaction to add arcs.
    Since the mass of product is greater than
    that of each reactant, it adds arcs by
    addArc(source=reactants, destination=product). 
    :param Reaction reaction:
    """
    if reaction.category != cn.REACTION_n_1:
      pass
    else:
      destination = [reaction.products[0].molecule]
      source = [reactant.molecule for reactant in reaction.reactants]
      self.addArc(source, destination, reaction)
      self.identifier = self.makeId()
    
  def addArc(self, source, destination, reaction=None):
    """
    Add arcs (edges) using two molecule lists (source/destination).
    :param list-Molecule source:
    :param list-Molecule destination:
    """
    arcs = itertools.product(source, destination)
    for arc in arcs:
      if not self.checkTypeOneError(arc, reaction):
        arc_source = self.getNode(arc[0])
        arc_destination = self.getNode(arc[1])
        # if there is already a preious reaction,
        if self.has_edge(arc_source, arc_destination):
          reaction_label = self.get_edge_data(arc_source, arc_destination)[REACTION]
          # if reaction.label is not already included in the attribute,
          if reaction.label not in set(reaction_label):
            reaction_label = reaction_label + [reaction.label]
        else:
          reaction_label = [reaction.label]
        self.add_edge(arc_source, arc_destination, reaction=reaction_label)
      else:
        continue
    
  def checkTypeOneError(self, arc, inequality_reaction=None):
    """
    Check Type I Error of an arc.
    If both source and destination are found
    in the same SOM, send error message and return True.
    If not, return False.
    :param tuple-Molecule arc:
    :param Reaction inequality_reaction:
    :return bool:
    """
    som1 = self.getNode(arc[0])
    som2 = self.getNode(arc[1])
    if som1 == som2:
      print("We have Type I Error...\n")
      print(arc[0], " and ", arc[1], " have the same weight by")
      for equality_reaction in list(som1.reactions):
        print(equality_reaction)
      print("\nHowever, reaction \"", inequality_reaction, 
            "\" implies ", arc[0], LESSTHAN, arc[1])
      print()
      return True
    else:
      return False
    
  def analyze(self, reactions):
    """
    Sort list of reactions and process them.
    Add arcs or sending error messages using
    checkTypeOneError or checkTypeTwoError.
    :param list-Reaction reactions:
    """
    # Associate the reaction category with the function
    # that processes that category
    reaction_dic = {
        cn.REACTION_1_1: self.processUniUniReaction,
        cn.REACTION_1_n: self.processUniMultiReaction,
        cn.REACTION_n_1: self.processMultiUniReaction,
        }
    # Process each type of reaction
    for category in reaction_dic.keys():
      for reaction in [r for r in reactions if r.category == category]:
        func = reaction_dic[category]
        func(reaction)
    #
    return self

In [23]:
m = MESGraph(simple)
m.nodes

NodeView(({FH4}, {serine}, {CH2FH4}, {HCHO}, {NADPH}, {CH3FH4}, {homocysteine}, {formate}, {ATP}, {CHOFH4}, {GAR}, {FGAR}, {AICAR}, {NADP}, {dUMP}, {FH2f}, {FFH2}, {MTX1}, {MTX2}, {MTX3}, {MTX4}, {MTX5}, {EMTX}, {DHFRf}, {MTX1b}, {MTX2b}, {MTX3b}, {MTX4b}, {MTX5b}))

In [24]:
m.analyze(simple.reactions)

We have Type I Error...

FH4  and  CH2FH4  have the same weight by
SHMTr: CH2FH4 -> FH4; cell * (Vm * (glycine / Km2 / (1 + glycine / Km2)) * (CH2FH4 / Km1) / (1 + CH2FH4 / Km1))
FFH2syn: FH2f -> FFH2; cell * Vm * FH2f
DHFReductase: FH2f -> FH4; cell * kter * FH2b

However, reaction " CH2FH4toHCHO: CH2FH4 -> FH4 + HCHO; cell * hl * CH2FH4 " implies  FH4  <  CH2FH4

We have Type I Error...

MTX1  and  MTX1b  have the same weight by
MTX2deg: MTX2b -> MTX2; cell * Vm * MTX2b
MTX3deg: MTX3b -> MTX3; cell * Vm * MTX3b
MTX4deg: MTX4b -> MTX4; cell * Vm * MTX4b
FPGS45: MTX4 -> MTX5; cell * Vm * MTX4
RFC: EMTX -> MTX1; ext * (Vm * EMTX / (Km + EMTX))
FPGS23: MTX2 -> MTX3; cell * Vm * MTX2
MTX5deg: MTX5b -> MTX5; cell * Vm * MTX5b
MTX1deg: MTX1b -> MTX1; cell * Vm * MTX1b
FPGS12: MTX1 -> MTX2; cell * Vm * MTX1
FPGS34: MTX3 -> MTX4; cell * Vm * MTX3

However, reaction " MTX1off: MTX1b -> MTX1 + DHFRf; cell * Vm * MTX1b " implies  MTX1  <  MTX1b

We have Type I Error...

MTX2  and  MTX2b  have th

{serine}->{CH2FH4, FFH2, FH2f, FH4}
{HCHO}->{CH2FH4, FFH2, FH2f, FH4}
{NADPH}->{CH3FH4}
{CH3FH4}->{CH2FH4, FFH2, FH2f, FH4}
{homocysteine}->{CH2FH4, FFH2, FH2f, FH4}
{formate}->{CHOFH4}
{ATP}->{CHOFH4}
{CHOFH4}->{CH2FH4, FFH2, FH2f, FH4}
{NADP}->{CHOFH4}
{dUMP}->{CH2FH4, FFH2, FH2f, FH4}
{DHFRf}->{EMTX, MTX1, MTX1b, MTX2, MTX2b, MTX3, MTX3b, MTX4, MTX4b, MTX5, MTX5b}
{CH2FH4, FFH2, FH2f, FH4}->{CH3FH4}
{CH2FH4, FFH2, FH2f, FH4}->{CHOFH4}
{AICAR, FGAR}->{CH2FH4, FFH2, FH2f, FH4}
{GAR}

In [25]:
list(m.nodes)[-1].reactions

{MTX2deg: MTX2b -> MTX2; cell * Vm * MTX2b,
 MTX3deg: MTX3b -> MTX3; cell * Vm * MTX3b,
 MTX4deg: MTX4b -> MTX4; cell * Vm * MTX4b,
 FPGS45: MTX4 -> MTX5; cell * Vm * MTX4,
 RFC: EMTX -> MTX1; ext * (Vm * EMTX / (Km + EMTX)),
 FPGS23: MTX2 -> MTX3; cell * Vm * MTX2,
 MTX5deg: MTX5b -> MTX5; cell * Vm * MTX5b,
 MTX1deg: MTX1b -> MTX1; cell * Vm * MTX1b,
 FPGS12: MTX1 -> MTX2; cell * Vm * MTX1,
 FPGS34: MTX3 -> MTX4; cell * Vm * MTX3}

In [21]:
reaction = simple.reactions[8]
print(reaction.category)
reaction

reaction_n_1


ATIC7: CHOFH4 + AICAR -> FH4; cell * (Vm * CHOFH4 * AICAR / (AICAR * CHOFH4 + CHOFH4 * Km2 + (AICAR + Km2) * Km1 * (1 + MTX1 / Ki21 + MTX2 / Ki22 + MTX3 / Ki23 + MTX4 / Ki24 + MTX5 / Ki25 + FH2f / Ki1 + FFH2 / Ki1f)))

In [232]:
for react in simple.reactions:
    print(react, "\n")

SHMT: FH4 + serine -> CH2FH4; cell * (Vm * (serine / Km2 / (1 + serine / Km2)) * (FH4 / Km1) / (1 + FH4 / Km1)) 

SHMTr: CH2FH4 -> FH4; cell * (Vm * (glycine / Km2 / (1 + glycine / Km2)) * (CH2FH4 / Km1) / (1 + CH2FH4 / Km1)) 

HCHOtoCH2FH4: FH4 + HCHO -> CH2FH4; cell * hp * FH4 * HCHO 

CH2FH4toHCHO: CH2FH4 -> FH4 + HCHO; cell * hl * CH2FH4 

MTHFR: CH2FH4 + NADPH -> CH3FH4; cell * (Vm * CH2FH4 * NADPH / (NADPH * CH2FH4 + CH2FH4 * Km2 + (NADPH + Km2) * Km1 * (1 + MTX1 / Ki21 + MTX2 / Ki22 + MTX3 / Ki23 + MTX4 / Ki24 + MTX5 / Ki25 + FH2f / Ki1))) 

MTR: CH3FH4 + homocysteine -> FH4; cell * (Vm * (homocysteine / Km2 / (1 + homocysteine / Km2)) * (CH3FH4 / Km1) / (1 + CH3FH4 / Km1)) 

HCOOHtoCHOFH4: FH4 + formate + ATP -> CHOFH4; cell * (Vm / ((1 + Km1 / FH4) * (1 + Km2 / ATP) * (1 + Km3 / formate))) 

GARFT: CHOFH4 + GAR -> FGAR + FH4; cell * (Vm * CHOFH4 * GAR / (GAR * CHOFH4 + CHOFH4 * Km2 + (GAR + Km2) * Km1 * (1 + MTX1 / Ki21 + MTX2 / Ki22 + MTX3 / Ki23 + MTX4 / Ki24 + MTX5 / Ki25 +

In [233]:
m.edges

OutEdgeView([])

In [234]:
unimulti_reaction = simple.reactions[3]
destination = [reactant.molecule for reactant in unimulti_reaction.reactants]
source = [produce.molecule for produce in unimulti_reaction.products]
unimulti_reaction

CH2FH4toHCHO: CH2FH4 -> FH4 + HCHO; cell * hl * CH2FH4

In [235]:
m.edges(data=True)

OutEdgeDataView([])

In [236]:
m.addArc(source, destination, unimulti_reaction)

In [237]:
m.edges(data=True)

OutEdgeDataView([({FH4}, {CH2FH4}, {'reaction': ['CH2FH4toHCHO']}), ({HCHO}, {CH2FH4}, {'reaction': ['CH2FH4toHCHO']})])

In [212]:
fh4 = simple.molecules[0]
ch2fh4 = simple.molecules[2]
print(fh4)
print(ch2fh4)

FH4
CH2FH4


In [213]:
m.has_edge(m.getNode(fh4), m.getNode(ch2fh4))

True

In [214]:
m.get_edge_data(m.getNode(fh4), m.getNode(ch2fh4))['reaction']

['CH2FH4toHCHO']

In [215]:
m.processUniMultiReaction(unimulti_reaction)
m.edges

OutEdgeView([({FH4}, {CH2FH4}), ({HCHO}, {CH2FH4})])

In [217]:
m.edges(data=True)

OutEdgeDataView([({FH4}, {CH2FH4}, {'reaction': ['CH2FH4toHCHO']}), ({HCHO}, {CH2FH4}, {'reaction': ['CH2FH4toHCHO']})])

In [218]:
multiuni_reaction = simple.reactions[0]
multiuni_reaction

SHMT: FH4 + serine -> CH2FH4; cell * (Vm * (serine / Km2 / (1 + serine / Km2)) * (FH4 / Km1) / (1 + FH4 / Km1))

In [219]:
m.processMultiUniReaction(multiuni_reaction)
m.edges

OutEdgeView([({FH4}, {CH2FH4}), ({serine}, {CH2FH4}), ({HCHO}, {CH2FH4})])

In [220]:
m.edges(data=True)

OutEdgeDataView([({FH4}, {CH2FH4}, {'reaction': ['CH2FH4toHCHO', 'SHMT']}), ({serine}, {CH2FH4}, {'reaction': ['SHMT']}), ({HCHO}, {CH2FH4}, {'reaction': ['CH2FH4toHCHO']})])

In [221]:
m.processUniUniReaction(reaction)

In [222]:
m.nodes

NodeView(({FH4}, {serine}, {CH2FH4}, {HCHO}, {NADPH}, {CH3FH4}, {homocysteine}, {formate}, {ATP}, {CHOFH4}, {GAR}, {FGAR}, {AICAR}, {NADP}, {dUMP}, {FH2f}, {FFH2}, {MTX1}, {MTX2}, {MTX3}, {MTX4}, {MTX5}, {EMTX}, {DHFRf}, {MTX1b}, {MTX2b}, {MTX3b}, {MTX4b}, {MTX5b}))

In [18]:
m.analyze(Reaction.reactions)
m.nodes

NodeView(({NAD}, {halfglucose}, {NADH}, {lactate}, {CoA}, {AcCoA}, {PO4}, {AcP}, {Ac}, {AcO}, {EtOH}, {Butanediol}, {O2}, {AcLac, AcetoinIn, AcetoinOut, pyruvate}, {ADP, ATP}))

In [19]:
m.edges

OutEdgeView([({NADH}, {NAD}), ({O2}, {NAD})])

In [20]:
m

{NADH}->{NAD}
{O2}->{NAD}
{halfglucose};{lactate};{CoA};{AcCoA};{PO4};{AcP};{Ac};{AcO};{EtOH};{Butanediol};{AcLac, AcetoinIn, AcetoinOut, pyruvate};{ADP, ATP}

In [453]:
m.edges

OutEdgeView([({Gly}, {DFG, E1, E2}), ({Gly}, {Mel}), ({MG}, {AA, Cn, FA, Glu, Man, _1DG, _3DG}), ({Fru}, {DFG, E1, E2}), ({AA, Cn, FA, Glu, Man, _1DG, _3DG}, {DFG, E1, E2}), ({AA, Cn, FA, Glu, Man, _1DG, _3DG}, {Mel})])

In [456]:
list(nx.isolates(m))

[]

In [424]:
identifier = ""
if m.edges:
    for edge in m.edges:
        identifier = identifier + str(edge[0]) + "->" + str(edge[1]) + "\n"
    
for key, node in enumerate(nx.isolates(m)):
    identifier = identifier + str(node)
    if key < (len(list(nx.isolates(m)))-1):
        identifier = identifier + cn.KINETICS_SEPARATOR
print(identifier)

{Gly}->{DFG, E1, E2}
{Gly}->{Mel}
{MG}->{AA, Cn, FA, Glu, Man, _1DG, _3DG}
{Fru}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{Mel}



{DFG};{E1};{E2};{Gly};{Cn};{_3DG};{FA};{_1DG};{AA};{Man};{Glu};{Mel};{MG};{Fru}

In [323]:
str(edge[0])

'{Gly}'

In [343]:
nodes = list(m.nodes)
print(nodes)

[{Gly}, {Mel}, {MG}, {Fru}, {DFG, E1, E2}, {AA, Cn, FA, Glu, Man, _1DG, _3DG}]


In [373]:
for x in nx.isolates(m):
    print(x)
    print(type(x))

{ATP, Ade, DNA, GTP, Gua, HX, IMP, PRPP, R5P, RNA, SAM, SAMP, UA, XMP, Xa, dATP, dGTP}
<class 'SBMLLint.games.som.SOM'>


In [341]:
nx.ancestors(m, list(m.edges)[0][1])

{{Fru}, {MG}, {Gly}, {AA, Cn, FA, Glu, Man, _1DG, _3DG}}

In [339]:
list(m.edges)[0][0]

{Gly}

In [374]:
m

;{ATP}+{HX}+{Ade}+{dATP}+{DNA}+{IMP}+{PRPP}+{RNA}+{SAMP}+{dGTP}+{Gua}+{GTP}+{XMP}+{Xa}+{SAM}+{R5P}+{UA}+

In [376]:
nx.isolates(m)

<generator object isolates.<locals>.<genexpr> at 0x1521ab2c50>

In [22]:
a = {1}.union({2})

In [23]:
a

{1, 2}

In [24]:
a.add(3)

In [25]:
a

{1, 2, 3}

In [26]:
SOM.soms

[{E2},
 {Gly},
 {Cn},
 {_3DG},
 {FA},
 {_1DG},
 {AA},
 {Man},
 {Glu},
 {Mel},
 {MG},
 {Fru}]